In [1]:
#sciezka z danymi
%cd "C:\Users\Dawid\Desktop\kurs\jdszr4-animalsi\wino"

C:\Users\Dawid\Desktop\kurs\jdszr4-animalsi\wino


In [61]:
#glowne biblioteki
import pandas as pd
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import *
from sklearn.model_selection import GridSearchCV 
from time import time 
from winsound import Beep

In [2]:
#wlasny modul
from dane.dane_wsadowe import *

lista zmiennych: 
 ['X', 'Y', 'X_train_all', 'X_test', 'Y_train_all', 'Y_test', 'X_train_reduced', 'X_val', 'Y_train_reduced', 'Y_val', 'X_train_all_scaled', 'X_test_scaled', 'X_val_scaled', 'X_train_reduced_scaled', 'X_train_all_scaled_PCA', 'X_test_scaled_PCA', 'X_train_reduced_scaled_PCA', 'X_val_scaled_PCA']


**1. Prosty model lasow losowych jako benchmark**

a) dane nieprzetworzone 

In [8]:
#wyjdzmy od prostego modelu na domyslnych parametrach
model_rf1 = RandomForestClassifier()
model_rf1.fit(X_train_reduced,Y_train_reduced)
pred_rf1 = model_rf1.predict(X_val)

In [11]:
#ocena jakosci modelu nr 1
rf1_cnf_matrix = confusion_matrix(Y_val, pred_rf1)
rf1_cnf_raport = classification_report(Y_val, pred_rf1)
print(rf1_cnf_raport)
print(rf1_cnf_matrix)

              precision    recall  f1-score   support

           1       0.00      0.00      0.00         7
           2       0.81      0.94      0.87       101
           3       0.45      0.25      0.32        20

    accuracy                           0.78       128
   macro avg       0.42      0.40      0.40       128
weighted avg       0.71      0.78      0.74       128

[[ 0  7  0]
 [ 0 95  6]
 [ 0 15  5]]


Ocena pierwszego modelu, model jak mozna bylo się spodziewać dobrze przewiduje klase najbardziej liczna, a w ogóle nie trafią w klase 1.

b) dane przeskalowane

In [23]:
#wyjdzmy od prostego modelu na domyslnych parametrach
model_rf2 = RandomForestClassifier()
model_rf2.fit(X_train_reduced_scaled,Y_train_reduced)
pred_rf2 = model_rf2.predict(X_val_scaled)

In [24]:
#ocena jakosci modelu nr 1
rf2_cnf_matrix = confusion_matrix(Y_val, pred_rf2)
rf2_cnf_raport = classification_report(Y_val, pred_rf2)
print(rf2_cnf_raport)
print(rf2_cnf_matrix)

              precision    recall  f1-score   support

           1       0.00      0.00      0.00         7
           2       0.82      0.93      0.87       101
           3       0.50      0.35      0.41        20

    accuracy                           0.79       128
   macro avg       0.44      0.43      0.43       128
weighted avg       0.73      0.79      0.75       128

[[ 0  7  0]
 [ 0 94  7]
 [ 0 13  7]]


C:\Users\Dawid\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Przy przeskalowanych danych poprawił nam się nieznacznie współczynik F1 dla klasy 3, ale wciąż nie udało nam się sklasyfikować żadnej obserwacji jako klasa 1.

c) dane PCA

In [25]:
#wyjdzmy od prostego modelu na domyslnych parametrach
model_rf3 = RandomForestClassifier()
model_rf3.fit(X_train_reduced_scaled_PCA,Y_train_reduced)
pred_rf3 = model_rf3.predict(X_val_scaled_PCA)

In [26]:
#ocena jakosci modelu nr 1
rf3_cnf_matrix = confusion_matrix(Y_val, pred_rf3)
rf3_cnf_raport = classification_report(Y_val, pred_rf3)
print(rf3_cnf_raport)
print(rf3_cnf_matrix)

              precision    recall  f1-score   support

           1       1.00      0.14      0.25         7
           2       0.82      0.93      0.87       101
           3       0.46      0.30      0.36        20

    accuracy                           0.79       128
   macro avg       0.76      0.46      0.50       128
weighted avg       0.78      0.79      0.76       128

[[ 1  6  0]
 [ 0 94  7]
 [ 0 14  6]]


Ten wariant w końcu sklasyfikował nam cos poprawnie dla klasy 1, ale w większości błędnie.

**2. Znajdzmy najlepsze parametry dla tego modelu**

In [53]:
#zacznijmy od małego zestawu, by dostrzec czy jest w ogóle jakas poprawa
params_rf1 = {"max_depth"        :list(range(2, 15, 2)),
              "n_estimators"     :list(range(50,211,40)),
              "min_samples_leaf" :list(range(2, 21, 3)),
              "max_features"     :[3,6,9,None]
}

In [71]:
r=  0   # policzymy ile jest mozliwych kombinacji parametrow
for j in params_rf1.values():
    s=len(j)
    r += s
r

23

a) model oparty na nieprzetworzonych danych

In [67]:
#zdefijmy objekt sprawdzajacy nasze parametry
grid_search1 =  GridSearchCV(RandomForestClassifier(),param_grid = params_rf1,cv =5,verbose = 1,njobs = 1,scoring = make_scorer(f1_score, average = "macro")).fit(X_train_all,Y_train_all)
Beep(400,1000)

In [80]:
# sprawdzmy nasz najlepszy zestaw parametrow
print(grid_search1.best_params_)

model_rf4 = grid_search1.best_estimator_
model_rf4.fit(X_train_all,Y_train_all)

pred_rf4 = model_rf4.predict(X_test)

{'max_depth': 14, 'max_features': 3, 'min_samples_leaf': 2, 'n_estimators': 50}


In [70]:
#ocena jakosci modelu z wybranymi najlepszymi parametrami
rf4_cnf_matrix = confusion_matrix(Y_test, pred_rf4)
rf4_cnf_raport = classification_report(Y_test, pred_rf4)
print(rf4_cnf_raport)
print(rf4_cnf_matrix)

              precision    recall  f1-score   support

           1       1.00      0.09      0.17        11
           2       0.89      0.96      0.92       263
           3       0.69      0.54      0.61        46

    accuracy                           0.87       320
   macro avg       0.86      0.53      0.57       320
weighted avg       0.87      0.87      0.85       320

[[  1  10   0]
 [  0 252  11]
 [  0  21  25]]


Nasz model znacznie poprawił precyzje dla klasy 2 i 3, jednak wciaz słabo klasyfikuje klase 1szą. 

b) model oparty na przeskalowanych danych

In [74]:
#zdefiniujmy obiek sprawdzajacy nasze parametry
grid_search2 =  GridSearchCV(RandomForestClassifier(),param_grid = params_rf1,cv =5,verbose = 1,n_jobs = 2,scoring = make_scorer(f1_score, average = "macro")).fit(X_train_all_scaled,Y_train_all)
Beep(400,1000)

Fitting 5 folds for each of 980 candidates, totalling 4900 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:   14.2s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:   50.0s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:  1.9min
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:  3.6min
[Parallel(n_jobs=2)]: Done 1246 tasks      | elapsed:  6.0min
[Parallel(n_jobs=2)]: Done 1796 tasks      | elapsed:  9.1min
[Parallel(n_jobs=2)]: Done 2446 tasks      | elapsed: 13.1min
[Parallel(n_jobs=2)]: Done 3196 tasks      | elapsed: 18.4min
[Parallel(n_jobs=2)]: Done 4046 tasks      | elapsed: 24.9min
[Parallel(n_jobs=2)]: Done 4900 out of 4900 | elapsed: 31.5min finished


In [75]:
# sprawdzmy nasz najlepszy zestaw parametrow
print(grid_search2.best_params_)

model_rf5 = grid_search2.best_estimator_
model_rf5.fit(X_train_all_scaled,Y_train_all)

pred_rf5 = model_rf5.predict(X_test_scaled)

{'max_depth': 14, 'max_features': None, 'min_samples_leaf': 2, 'n_estimators': 130}


In [76]:
#ocena jakosci modelu z wybranymi najlepszymi parametrami
rf5_cnf_matrix = confusion_matrix(Y_test, pred_rf5)
rf5_cnf_raport = classification_report(Y_test, pred_rf5)
print(rf5_cnf_raport)
print(rf5_cnf_matrix)

              precision    recall  f1-score   support

           1       0.50      0.09      0.15        11
           2       0.89      0.94      0.92       263
           3       0.64      0.54      0.59        46

    accuracy                           0.86       320
   macro avg       0.68      0.53      0.55       320
weighted avg       0.84      0.86      0.84       320

[[  1  10   0]
 [  1 248  14]
 [  0  21  25]]


Przy przeskalowanych danych nie widzimy poprawy, wręcz delikatne pogorszenie.

c) dane PCA

In [77]:
#zdefiniujmy obiek sprawdzajacy nasze parametry
grid_search3 =  GridSearchCV(RandomForestClassifier(),param_grid = params_rf1,cv =5,verbose = 1,n_jobs = 3,scoring = make_scorer(f1_score, average = "macro")).fit(X_train_all_scaled_PCA,Y_train_all)
Beep(400,1000)

Fitting 5 folds for each of 980 candidates, totalling 4900 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:   12.7s
[Parallel(n_jobs=3)]: Done 194 tasks      | elapsed:   47.3s
[Parallel(n_jobs=3)]: Done 444 tasks      | elapsed:  1.6min
[Parallel(n_jobs=3)]: Done 794 tasks      | elapsed:  2.8min
[Parallel(n_jobs=3)]: Done 1244 tasks      | elapsed:  4.3min
[Parallel(n_jobs=3)]: Done 1794 tasks      | elapsed:  7.7min
[Parallel(n_jobs=3)]: Done 2444 tasks      | elapsed: 10.9min
[Parallel(n_jobs=3)]: Done 3194 tasks      | elapsed: 14.4min
[Parallel(n_jobs=3)]: Done 4044 tasks      | elapsed: 18.2min
[Parallel(n_jobs=3)]: Done 4900 out of 4900 | elapsed: 23.0min finished


In [85]:
# sprawdzmy nasz najlepszy zestaw parametrow
print(grid_search3.best_params_)

model_rf6 = grid_search3.best_estimator_
model_rf6.fit(X_train_all_scaled_PCA,Y_train_all)

pred_rf6 = model_rf6.predict(X_test_scaled_PCA)

{'max_depth': 14, 'max_features': None, 'min_samples_leaf': 2, 'n_estimators': 170}


In [86]:
#ocena jakosci modelu z wybranymi najlepszymi parametrami
rf6_cnf_matrix = confusion_matrix(Y_test, pred_rf6)
rf6_cnf_raport = classification_report(Y_test, pred_rf6)
print(rf6_cnf_raport)
print(rf6_cnf_matrix)

              precision    recall  f1-score   support

           1       0.00      0.00      0.00        11
           2       0.89      0.94      0.91       263
           3       0.63      0.52      0.57        46

    accuracy                           0.85       320
   macro avg       0.51      0.49      0.49       320
weighted avg       0.82      0.85      0.83       320

[[  0  10   1]
 [  2 248  13]
 [  0  22  24]]


Nasz model przy danych po PCA odnotował gorsze wyniki niż bez skalowania.

**3. Popracujmy nad niezbalansowaniem klas**

In [87]:
#znajdzmy najpierw dla danych nieprzeskalowanych mozliwie lepszy zestaw parametrow, 
#na podstawie wnioskow z juz przeprowadzonych obliczen.

params_rf2 = {"max_depth"        :list(range(14, 25, 2)),
              "n_estimators"     :list(range(50,221,40)),
              "min_samples_leaf" :list(range(2, 6)),
              "max_features"     :[3,4,9,11,None]              
}

In [88]:
#zdefiniujmy obiek sprawdzajacy nasze parametry
grid_search4 =  GridSearchCV(RandomForestClassifier(),param_grid = params_rf2,cv =5,verbose = 1,n_jobs = 3,scoring = make_scorer(f1_score, average = "macro")).fit(X_train_all,Y_train_all)
Beep(400,1000)

Fitting 5 folds for each of 600 candidates, totalling 3000 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:   15.4s
[Parallel(n_jobs=3)]: Done 194 tasks      | elapsed:   59.8s
[Parallel(n_jobs=3)]: Done 444 tasks      | elapsed:  3.2min
[Parallel(n_jobs=3)]: Done 794 tasks      | elapsed:  5.5min
[Parallel(n_jobs=3)]: Done 1244 tasks      | elapsed:  8.7min
[Parallel(n_jobs=3)]: Done 1794 tasks      | elapsed: 12.5min
[Parallel(n_jobs=3)]: Done 2444 tasks      | elapsed: 17.3min
[Parallel(n_jobs=3)]: Done 3000 out of 3000 | elapsed: 21.3min finished


In [89]:
# sprawdzmy nasz najlepszy zestaw parametrow
print(grid_search4.best_params_)

model_rf7 = grid_search4.best_estimator_
model_rf7.fit(X_train_all,Y_train_all)

pred_rf7 = model_rf7.predict(X_test)

{'max_depth': 14, 'max_features': 11, 'min_samples_leaf': 2, 'n_estimators': 90}


In [90]:
#ocena jakosci modelu z wybranymi najlepszymi parametrami
rf7_cnf_matrix = confusion_matrix(Y_test, pred_rf7)
rf7_cnf_raport = classification_report(Y_test, pred_rf7)
print(rf7_cnf_raport)
print(rf7_cnf_matrix)

              precision    recall  f1-score   support

           1       0.50      0.09      0.15        11
           2       0.89      0.95      0.92       263
           3       0.66      0.54      0.60        46

    accuracy                           0.86       320
   macro avg       0.68      0.53      0.56       320
weighted avg       0.84      0.86      0.84       320

[[  1  10   0]
 [  1 249  13]
 [  0  21  25]]


Wykorzystujac pelna ilosc zmiennych w naszym modelu poprawilismy klasyfikacje dla klasy 3, ale to nadal nie jest idealna klasyfikacja dla klasy 3 i slaba dla klasy 1.

In [93]:
# dobierzmy jeszcze do naszego najlepszego modelu najbardziej optymalny zestaw wag
weights = np.linspace(0.05,0.95,50)
params_weight1 = {"class_weight": [{1:x, 2:y,3:1-x-y} for x in weights for y in weights  if x+y <1]}

In [94]:
#zdefiniujmy obiekt sprawdzajacy nasze losowane wagi na ostatnim wyliczonym modelu
grid_search5 =  GridSearchCV(grid_search4.best_estimator_,param_grid = params_weight1,cv =5,verbose = 1,n_jobs = 3,scoring = make_scorer(f1_score, average = "macro")).fit(X_train_all,Y_train_all)
Beep(400,1000)

Fitting 5 folds for each of 1225 candidates, totalling 6125 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:   20.1s
[Parallel(n_jobs=3)]: Done 194 tasks      | elapsed:  1.3min
[Parallel(n_jobs=3)]: Done 444 tasks      | elapsed:  3.0min
[Parallel(n_jobs=3)]: Done 794 tasks      | elapsed:  5.1min
[Parallel(n_jobs=3)]: Done 1244 tasks      | elapsed:  7.8min
[Parallel(n_jobs=3)]: Done 1794 tasks      | elapsed: 11.2min
[Parallel(n_jobs=3)]: Done 2444 tasks      | elapsed: 15.2min
[Parallel(n_jobs=3)]: Done 3194 tasks      | elapsed: 20.1min
[Parallel(n_jobs=3)]: Done 4044 tasks      | elapsed: 25.6min
[Parallel(n_jobs=3)]: Done 4994 tasks      | elapsed: 32.0min
[Parallel(n_jobs=3)]: Done 6044 tasks      | elapsed: 39.3min
[Parallel(n_jobs=3)]: Done 6125 out of 6125 | elapsed: 39.8min finished


In [95]:
# sprawdzmy jaki jest nasz najlepszy zestaw wag
print(grid_search5.best_params_)

model_rf8 = grid_search5.best_estimator_
model_rf8.fit(X_train_all,Y_train_all)

pred_rf8 = model_rf8.predict(X_test)

{'class_weight': {1: 0.7295918367346939, 2: 0.05, 3: 0.22040816326530616}}


In [96]:
#ocena jakosci modelu z wybranymi najlepszymi wagami i parametrami
rf8_cnf_matrix = confusion_matrix(Y_test, pred_rf8)
rf8_cnf_raport = classification_report(Y_test, pred_rf8)
print(rf8_cnf_raport)
print(rf8_cnf_matrix)

              precision    recall  f1-score   support

           1       0.20      0.09      0.13        11
           2       0.89      0.92      0.90       263
           3       0.59      0.57      0.58        46

    accuracy                           0.84       320
   macro avg       0.56      0.52      0.54       320
weighted avg       0.82      0.84      0.83       320

[[  1  10   0]
 [  4 241  18]
 [  0  20  26]]


Dziwne, ale model zaproponował nam nielogiczne rozłozenie wag per klasa, wpłynęło to pozytywnie tylko na klasę 2, pozostałe klasy bez zmian.
Pozostaje nam tylko sprawdzic pole pod krzywa AUC

In [101]:
grid_search5.best_estimator_

RandomForestClassifier(class_weight={1: 0.7295918367346939, 2: 0.05,
                                     3: 0.22040816326530616},
                       max_depth=14, max_features=11, min_samples_leaf=2,
                       n_estimators=90)

**4. Pole pod krzywą ROC**   
- próba znalezienia wydajniejszej klasyfikacji przy zmianie progu klasyfikacji

In [115]:
#rozważmy wyniki dwóch najlepszych modeli i poszukajmy bardziej optymalnych progów
# model_rf4 i model_rf8

Y_pred_rf4_proba_c1 = model_rf4.predict_proba(X_test)[::,0]
Y_pred_rf4_proba_c2 = model_rf4.predict_proba(X_test)[::,1]
Y_pred_rf4_proba_c3 = model_rf4.predict_proba(X_test)[::,2]

In [117]:
#wykreslenie krzywej ROC

FPR, TPR, tresholds = roc_curve(Y_test, Y_pred_rf4_proba_c1)

ValueError: multiclass format is not supported

In [118]:
Y_test

417     2
1506    2
991     2
638     3
1230    2
       ..
491     3
1334    2
1321    2
386     2
1223    2
Name: quality_category, Length: 320, dtype: int64

In [ ]:
#zrob AUC - jak to zrobic dla 3 klas, jak przypisac pary, wagi??? - Michała cisnij !!!